# Global StreetScapes

In [130]:
from utils import download_from_huggingface
import pandas as pd
import duckdb
import os


## Data Extraction

In [131]:
# Download Global StreetScapes data from Hugging Face
# The entire folder is approximately 37GB. You can choose to download specific files instead.
repo_id = "NUS-UAL/global-streetscapes"
repo_type = "dataset"
folder_path = "data/"
local_dir_folder = "../../data/0-raw/Global-StreetScapes/"
local_dir_files = "../../data/0-raw/Global-StreetScapes/info"
file_list = ["cities688.csv", "info.csv"]

# Download the entire 'data' folder directly into the specified local directory
download_from_huggingface(repo_id, repo_type, folder_path=folder_path, local_dir=local_dir_folder)

# Download specific files into the 'info' directory
download_from_huggingface(repo_id, repo_type, file_paths=file_list, local_dir=local_dir_files)


File 'climate.csv' already exists in the directory. Skipping download.
File 'contextual.csv' already exists in the directory. Skipping download.
File 'ephem.csv' already exists in the directory. Skipping download.
File 'gadm.csv' already exists in the directory. Skipping download.
File 'ghsl.csv' already exists in the directory. Skipping download.
File 'h3.csv' already exists in the directory. Skipping download.
File 'instances.csv' already exists in the directory. Skipping download.
File 'metadata_common_attributes.csv' already exists in the directory. Skipping download.
File 'metadata_kv.csv' already exists in the directory. Skipping download.
File 'metadata_mly1.csv' already exists in the directory. Skipping download.
File 'metadata_mly2.csv' already exists in the directory. Skipping download.
File 'metadata_mly3.csv' already exists in the directory. Skipping download.
File 'metadata_mly4.csv' already exists in the directory. Skipping download.
File 'metadata_mly5.csv' already exist

File 'cities688.csv' already exists in the directory. Skipping download.
File 'info.csv' already exists in the directory. Skipping download.


## Data exploration

In [132]:
# Let's select some cities where we have good amount of images and different climates
cities_df = pd.read_csv('../../data/0-raw/Global-StreetScapes/info/cities688.csv')

# Select the desired columns
selected_columns = ['city', 'img_count', 'continent', 'koppen_geiger_zone', 'zone_description']

# Create a list of cities to filter
selected_cities = ['Amsterdam', 'Barcelona', 'Berlin', 'London', 'Athens', 'Lisbon']

# Create a table of selected cities with the desired columns, ordered by img_count in descending order
cities_table = cities_df[cities_df['city'].isin(selected_cities)][selected_columns].sort_values('img_count', ascending=False)

# Display the selected cities
cities_table


city  img_count continent koppen_geiger_zone  \
456     Berlin     281033    Europe                Cfb   
396     Athens     106003    Europe                Csa   
321     Lisbon      87461    Europe                Csa   
368     London      41837    Europe                Cfb   
221  Amsterdam      26421    Europe                Cfb   
508  Barcelona      21218    Europe                Csa   

                   zone_description  
456  Marine west coast, warm summer  
396       Mediterranean, hot summer  
321       Mediterranean, hot summer  
368  Marine west coast, warm summer  
221  Marine west coast, warm summer  
508       Mediterranean, hot summer

In [133]:

# Read the info.csv file
info_df = pd.read_csv('../../data/0-raw/Global-StreetScapes/info/info.csv')

# Display only 'Filename' and 'Overview' columns where 'Filename' is not null
display_columns = ['Filename', 'Overview', 'Notes']
display_df = info_df[display_columns].dropna(subset=['Filename'])

# Reset the index and drop it to remove the index column
display_df = display_df.reset_index(drop=True)

# Set display options to show full content of 'Overview' column
pd.set_option('display.max_colwidth', None)

# Display the data
display(display_df)

# Reset display options to default
pd.reset_option('display.max_colwidth')

Filename  \
0                      climate.csv   
1                   contextual.csv   
2                        ephem.csv   
3                         gadm.csv   
4                         ghsl.csv   
5                           h3.csv   
6                    instances.csv   
7   metadata_common_attributes.csv   
8                  metadata_kv.csv   
9                metadata_mly1.csv   
10               metadata_mly2.csv   
11               metadata_mly3.csv   
12               metadata_mly4.csv   
13               metadata_mly5.csv   
14                         osm.csv   
15                  perception.csv   
16                   places365.csv   
17                      season.csv   
18                segmentation.csv   
19                  simplemaps.csv   
20                       speed.csv   

                                                                                                                                          Overview  \
0                                                                          Contains the Koppen climate zone associated with each image's location.   
1                                                                                Contains the eight contextual attributes inferred for each image.   
2                   Contains the temporal information of each image calculated using the python package 'PyEphem' with regards to the time of day.   
3                                                            Contains the administrative area associated with each image, at all available levels.   
4        Contains the degree of urbanisation associated with the location of the image, calculated using the Global Human Settlement Layer (GHSL).   
5                     Contains the ID of the h3-indexed hexagon associated with each image, at all available resolution levels from level 0 to 15.   
6                                                                          Contains the count of instances (65 categories) detected in each image.   
7         Contains the common basic metadata attributes that are provided by both Mapillary and KartaView, and those we computed for both sources.   
8                                                                              Contains the metadata of each SVI originally provided by KartaView.   
9                                                                              Contains the metadata of each SVI originally provided by Mapillary.   
10                                                                                                                                             NaN   
11                                                                                                                                             NaN   
12                                                                                                                                             NaN   
13                                                                                                                                             NaN   
14                                                  Contains the information of the OSM street found nearest to each image within its 10 m radius.   
15  Contains the scores, in scale of 0-10, predicted for each of six perceptual dimensions (Safe, Lively, Beautiful, Wealthy, Boring, Depressing).   
16                                                                                         Contains the place/scene classification for each image.   
17                                                                  Contains the season determined for each image based on climate zone and month.   
18                               Contains the count of segmented pixels of each semantic classes (65 categories in total) identified in the image.   
19                          Contains the information of the city associated with the image, obtained from the World Cities database by Simplemaps.   
20                                     

In [134]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Load the CSV and propagate Filename values
info_df = pd.read_csv('../../data/0-raw/Global-StreetScapes/info/info.csv').ffill()

# Set display options to show full content of all columns
pd.set_option('display.max_colwidth', None)

# Create a dropdown for Filename selection with an "All" option
filename_dropdown = widgets.Dropdown(
    options=['Select a filename'] + info_df['Filename'].unique().tolist(),
    value='Select a filename',  # Default to the indicator value
    description='Filename:',
    layout=widgets.Layout(width='30%')
)

# Create a search text box for Field filtering without a dropdown
field_search = widgets.Text(
    placeholder='Type to search fields...',
    description='Field:',
    layout=widgets.Layout(width='30%')
)

output = widgets.Output()

# Function to filter and display data based on selected filename and field
def filter_data(change=None):
    with output:
        clear_output(wait=True)
        
        # If the placeholder option is selected, show a message
        if filename_dropdown.value == 'Select a filename':
            display(HTML("<p><b>Please select a filename to view the data.</b></p>"))
            return
        
        # Filter the data
        filtered_df = info_df.copy()
        
        # Filter by Filename if not "All"
        if filename_dropdown.value != 'All':
            filtered_df = filtered_df[filtered_df['Filename'] == filename_dropdown.value]

        # Filter by field if there is text in the field_search box
        if field_search.value.strip():
            filtered_df = filtered_df[filtered_df['Field'].str.contains(field_search.value, case=False, na=False)]

        # Display the filtered data with adjusted column widths
        if not filtered_df.empty:
            html_table = filtered_df[['Filename', 'Field', 'Format', 'Explanation']].to_html(index=False)
            styled_table = f"""
            <style>
            table {{width: 100%;}}
            th, td {{
                word-wrap: break-word;
                max-width: 700px;
                overflow: hidden;
                text-overflow: ellipsis;
            }}
            th:nth-child(1), td:nth-child(1) {{max-width: 120px;}}  /* Filename column */
            th:nth-child(4), td:nth-child(4) {{max-width: 600px;}}  /* Explanation column */
            </style>
            {html_table}
            """
            display(HTML(styled_table))
        else:
            display(HTML("<p>No results found.</p>"))

# Attach functions to widget events
filename_dropdown.observe(filter_data, names='value')
field_search.observe(filter_data, names='value')

# Display widgets
widget_box = widgets.HBox([filename_dropdown, field_search])
display(widget_box, output)

# Initial message to guide the user
filter_data()

# Reset display options to default
pd.reset_option('display.max_colwidth')

Output()

## Data Manipulation